In [1]:
!pip install -q  torch peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 accelerate  datasets #xformers
import transformers
import torch
import datasets
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
from peft import LoraConfig
from trl import SFTTrainer
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00


## **Load dataset from HuggingFace**

In [2]:
# Dataset
data_name = "jiuyuan/policy_AI"
training_data = load_dataset(data_name, split="train")
test_data = load_dataset(data_name, split="test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/75 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/19 [00:00<?, ? examples/s]

In [3]:
training_data

Dataset({
    features: ['input', 'output', '__index_level_0__'],
    num_rows: 75
})

In [5]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [6]:
# Model and tokenizer names
base_model_name = "meta-llama/Llama-2-7b-chat-hf"
refined_model = "llama-2-7b-policyAI"

# Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"  # Fix for fp16

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map={"": 0}
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [7]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['input'])):
        text = f"### Question: {example['input'][i]}\n Briefly, in 100 words answer the question. ### Answer: {example['output'][i]}</s>"
        output_texts.append(text)
    return output_texts


## **Fine tune**

In [8]:
# LoRA Config
peft_parameters = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=16,#rank
    bias="none",
    task_type="CAUSAL_LM"
)

# batch_size_multiplier = 2
accumulation_multiplier = 2
# Training Params
train_params = TrainingArguments(
    output_dir="./results_modified",
    evaluation_strategy = "epoch",
    num_train_epochs=10*accumulation_multiplier,
    per_device_train_batch_size=4*1,
    gradient_accumulation_steps=1 * accumulation_multiplier,
    optim="paged_adamw_32bit",
    save_steps=100,
    logging_steps=20//accumulation_multiplier,
    learning_rate=1e-5,
    weight_decay=2e-3,
    fp16=False,
    bf16=False,
    max_grad_norm=0.1,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

# Trainer
fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=training_data,
    eval_dataset = test_data,
    formatting_func=formatting_prompts_func, #hereeee
    peft_config=peft_parameters,
    # dataset_text_field="text",
    tokenizer=llama_tokenizer,
    args=train_params
)

# Training
fine_tuning.train()

# Save Model
fine_tuning.model.save_pretrained(refined_model)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/75 [00:00<?, ? examples/s]

Map:   0%|          | 0/19 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,No log,2.752605
2,2.873800,2.708901
2,2.824000,2.663248
4,2.790900,2.605659
4,2.734300,2.549197
6,2.652700,2.483747
6,2.605400,2.424042
8,2.554600,2.358225
8,2.495000,2.300764
10,2.398600,2.240205


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


In [7]:
# from google.colab import drive
# drive.mount('/content/drive')

In [8]:
# fine_tuning.model.save_pretrained("/content/drive/MyDrive/94812/llama-2-7b-policyAI")

In [9]:
# # fine_tuning is the trainer
# model_path = "/content/drive/MyDrive/94812/llama-2-7b-policyAI"
# fine_tuning.save_model(model_path)
# fine_tuning.model.config.save_pretrained(model_path+"config.json")

In [10]:
# model_path = "/content/drive/MyDrive/94812/llama-2-7b-policyAI"

# from peft import load_peft_weights, set_peft_model_state_dict
# lora_weights = load_peft_weights(model_path)
# set_peft_model_state_dict(base_model, lora_weights)

## **Generations & Evaluation**

In [9]:
df_test_all=test_data.to_pandas()
df_train=training_data.to_pandas()
df_test_all

,input,output,__index_level_0__
0,Why is international cooperation on AI important?,International cooperation on AI is crucial due...,70
1,Can you describe the approach taken by the res...,The research team prepared generic interview p...,21
2,What concerns do critics have regarding the EU...,"Critics, including EU businesses, have express...",37
3,How does the European Union classify AI system...,The European Union's AI Act classifies AI syst...,36
4,What are the Universal Guidelines for Artifici...,The Universal Guidelines include rights to tra...,58
5,What role do whistleblowers and complaints pla...,Whistleblowers and individual complaints are c...,48
6,How's UK's approach different from that of the...,"Unlike the European Union (EU), the UK’s appro...",14
7,How does Japan's approach to AI regulation dif...,"Japan focuses on a risk-based, agile, and mult...",31
8,"What are foundational AI models, and why are t...","Foundational AI models, such as large language...",60
9,What additional measures are needed for intern...,The article suggests that more ambitious trade...,64


In [10]:
df_test_all['output'].apply(lambda x:len(x.split())).describe(), df_train['output'].apply(lambda x:len(x.split())).describe()

(count    19.000000
 mean     40.631579
 std      11.591235
 min      17.000000
 25%      33.000000
 50%      41.000000
 75%      48.500000
 max      61.000000
 Name: output, dtype: float64,
 count      75.000000
 mean       73.893333
 std       136.433092
 min        21.000000
 25%        31.000000
 50%        38.000000
 75%        52.500000
 max      1020.000000
 Name: output, dtype: float64)

In [11]:
from tqdm.notebook import tqdm
import gc
generations =[]
for i in tqdm(range(len(df_test_all)), "generating..."):
  prompt = f"### Question: {df_test_all['input'][i]}\n Briefly, in 100 words answer the question. ### Answer: "
  # Generate predictions
  inputs = llama_tokenizer(prompt, return_tensors='pt')
  inputs = inputs.to("cuda")
  output = fine_tuning.model.generate(**inputs, max_new_tokens=150,temperature=0.2)
  response = llama_tokenizer.decode(output[0].tolist())
  # print(response)
  # break
  generations.append(response)
  del inputs, output
  gc.collect()
  torch.cuda.empty_cache()

generations

generating...:   0%|          | 0/19 [00:00<?, ?it/s]

['<s> ### Question: Why is international cooperation on AI important?\n Briefly, in 100 words answer the question. ### Answer:  International cooperation on AI is important because it can help address global challenges, such as climate change, pandemics, and economic inequality, more effectively than any one country could alone. It can also help ensure that AI is developed and used in ways that are ethical, transparent, and respectful of human rights.</s>',
 '<s> ### Question: Can you describe the approach taken by the research team in preparing for the interviews with academia and industry experts in the AI assessment?\n Briefly, in 100 words answer the question. ### Answer:  The research team took a systematic approach to prepare for the interviews with academia and industry experts in the AI assessment. They first conducted a comprehensive literature review to identify key themes and trends in AI research and development. Then, they developed a set of interview questions that aligne

In [12]:
df_test_all['fine_tuned_llama_generations']=generations
# df_test_all.to_csv('results.csv')

In [13]:
# Model
raw_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map={"": 0}
)
raw_model.config.use_cache = False
raw_model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## **Raw model generations**

In [14]:
from tqdm.notebook import tqdm

import gc
generations_original =[]
for i in tqdm(range(len(df_test_all)), "generating..."):
  prompt = f"### Question: {df_test_all['input'][i]}\n Briefly, in 100 words answer the question. ### Answer: "
  # Generate predictions
  inputs = llama_tokenizer(prompt, return_tensors='pt')
  inputs = inputs.to("cuda")
  # pipline = transformers.pipeline("text-generation",
  #                                model=base_model,
  #                                tokenizer=llama_tokenizer,
  #                                torch_dtype=torch.float16,
  #                                )
  # output = pipeline(prompt,temperature=0.2,
  #                               top_k=50,
  #                               top_p=0.9,
  #                    max_length=500)
  # print(output)
  output = raw_model.generate(**inputs, max_new_tokens=100,temperature=0.2)
  response = llama_tokenizer.decode(output[0].tolist())
  # print(response)
  # break
  generations_original.append(response)
  del inputs, output
  gc.collect()
  torch.cuda.empty_cache()

generations_original

generating...:   0%|          | 0/19 [00:00<?, ?it/s]

['<s> ### Question: Why is international cooperation on AI important?\n Briefly, in 100 words answer the question. ### Answer:  International cooperation on AI is important because it allows countries to pool their resources, share knowledge, and develop standards and regulations that promote responsible and ethical AI development and use. This can help ensure that AI is developed and used in ways that benefit all countries and avoid negative consequences such as job displacement, privacy violations, and exacerbation of social inequalities. Additionally, international cooperation can help address global challenges such as climate change, health pandem',
 '<s> ### Question: Can you describe the approach taken by the research team in preparing for the interviews with academia and industry experts in the AI assessment?\n Briefly, in 100 words answer the question. ### Answer:  The research team took a multi-faceted approach in preparing for the interviews with academia and industry experts

In [15]:
df_test_all['original_llama_generations']=generations_original
df_test_all

,input,output,__index_level_0__,fine_tuned_llama_generations,original_llama_generations
0,Why is international cooperation on AI important?,International cooperation on AI is crucial due...,70,<s> ### Question: Why is international coopera...,<s> ### Question: Why is international coopera...
1,Can you describe the approach taken by the res...,The research team prepared generic interview p...,21,<s> ### Question: Can you describe the approac...,<s> ### Question: Can you describe the approac...
2,What concerns do critics have regarding the EU...,"Critics, including EU businesses, have express...",37,<s> ### Question: What concerns do critics hav...,<s> ### Question: What concerns do critics hav...
3,How does the European Union classify AI system...,The European Union's AI Act classifies AI syst...,36,<s> ### Question: How does the European Union ...,<s> ### Question: How does the European Union ...
4,What are the Universal Guidelines for Artifici...,The Universal Guidelines include rights to tra...,58,<s> ### Question: What are the Universal Guide...,<s> ### Question: What are the Universal Guide...
5,What role do whistleblowers and complaints pla...,Whistleblowers and individual complaints are c...,48,<s> ### Question: What role do whistleblowers ...,<s> ### Question: What role do whistleblowers ...
6,How's UK's approach different from that of the...,"Unlike the European Union (EU), the UK’s appro...",14,<s> ### Question: How's UK's approach differen...,<s> ### Question: How's UK's approach differen...
7,How does Japan's approach to AI regulation dif...,"Japan focuses on a risk-based, agile, and mult...",31,<s> ### Question: How does Japan's approach to...,<s> ### Question: How does Japan's approach to...
8,"What are foundational AI models, and why are t...","Foundational AI models, such as large language...",60,<s> ### Question: What are foundational AI mod...,<s> ### Question: What are foundational AI mod...
9,What additional measures are needed for intern...,The article suggests that more ambitious trade...,64,<s> ### Question: What additional measures are...,<s> ### Question: What additional measures are...


In [16]:
df_test_all['fine_tuned_llama_generations']=generations
def extract_text_between_tags(text):
    # Define the regex pattern to match text between </s> tags

    import re
    pattern = r'### Answer:(.*?)(</s>|$)'

    # Use re.findall() to find all occurrences
    matches = re.findall(pattern, text)[0][0]
    print(matches)
    return matches
df_test_all['original_llama_generations']= df_test_all['original_llama_generations'].apply(lambda x: x.split('Answer:', 1)[-1] if 'Answer:' in x else x)
df_test_all['original_llama_generations']= df_test_all['original_llama_generations'].apply(lambda x: x.replace('</s>', '').replace('<s>', '').replace('</s>s', ''))
df_test_all['fine_tuned_llama_generations']= df_test_all['fine_tuned_llama_generations'].apply(lambda x: extract_text_between_tags(x.replace('\n', ' ')))
df_test_all['fine_tuned_llama_generations']= df_test_all['fine_tuned_llama_generations'].apply(lambda x: x.replace('</s>', '').replace('<s>', '').replace('</s>s', ''))
df_test_all

  International cooperation on AI is important because it can help address global challenges, such as climate change, pandemics, and economic inequality, more effectively than any one country could alone. It can also help ensure that AI is developed and used in ways that are ethical, transparent, and respectful of human rights.
  The research team took a systematic approach to prepare for the interviews with academia and industry experts in the AI assessment. They first conducted a comprehensive literature review to identify key themes and trends in AI research and development. Then, they developed a set of interview questions that aligned with these themes and trends. Finally, they conducted a pilot interview to refine and improve the interview questions. The team also established a set of criteria for evaluating the quality of the interviews and assessing the credibility of the experts. 
  Critics argue that the EU's AI Act could stifle innovation by imposing overly broad and vague d

,input,output,__index_level_0__,fine_tuned_llama_generations,original_llama_generations
0,Why is international cooperation on AI important?,International cooperation on AI is crucial due...,70,International cooperation on AI is important...,International cooperation on AI is important...
1,Can you describe the approach taken by the res...,The research team prepared generic interview p...,21,The research team took a systematic approach...,The research team took a multi-faceted appro...
2,What concerns do critics have regarding the EU...,"Critics, including EU businesses, have express...",37,Critics argue that the EU's AI Act could sti...,\nCritics argue that the EU's AI Act could st...
3,How does the European Union classify AI system...,The European Union's AI Act classifies AI syst...,36,The European Union classifies AI systems und...,\nThe European Union classifies AI systems un...
4,What are the Universal Guidelines for Artifici...,The Universal Guidelines include rights to tra...,58,The Universal Guidelines for Artificial Inte...,EPIC (Electronic Privacy Information Center)...
5,What role do whistleblowers and complaints pla...,Whistleblowers and individual complaints are c...,48,Whistleblowers and complaints play a crucial...,Whistleblowers and complaints play a crucial...
6,How's UK's approach different from that of the...,"Unlike the European Union (EU), the UK’s appro...",14,The UK's approach is different from the EU's...,\nThe UK's approach to data protection is dif...
7,How does Japan's approach to AI regulation dif...,"Japan focuses on a risk-based, agile, and mult...",31,Japan's approach to AI regulation focuses on...,Japan's approach to AI regulation differs fr...
8,"What are foundational AI models, and why are t...","Foundational AI models, such as large language...",60,Foundational AI models are the basic buildin...,Foundational AI models are the basic buildin...
9,What additional measures are needed for intern...,The article suggests that more ambitious trade...,64,100 words: International cooperation on found...,\nInternational cooperation on foundational A...


In [17]:
df_test_all.to_csv('results.csv')

## **Metrics**

In [18]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install nltk rouge -q
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge


In [23]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
def jaccard_similarity(text1, text2):
    """Calculates the Jaccard Similarity between two texts."""
    # Tokenize both texts
    tokens1 = set(text1.lower().split())
    tokens2 = set(text2.lower().split())

    intersection = tokens1.intersection(tokens2)
    union = tokens1.union(tokens2)
    similarity = len(intersection) / len(union)
    return similarity
def calculate_bleu_rouge(reference, candidate):
    """
    Calculate BLEU and ROUGE scores for a pair of strings.

    :param reference: The reference string (ground truth).
    :param candidate: The candidate string (generated text).
    :return: A dictionary containing BLEU and ROUGE scores.
    """
    # Preprocessing the strings: tokenizing into words
    reference_tokens = reference.split()
    candidate_tokens = candidate.split()

    # Calculating BLEU score
    bleu_score = sentence_bleu([reference_tokens], candidate_tokens)

    # Calculating ROUGE score
    rouge = Rouge()
    rouge_score = rouge.get_scores(candidate, reference)[0]

    return np.array([ bleu_score,rouge_score['rouge-1']['r'], rouge_score['rouge-2']['r'], rouge_score['rouge-l']['r']], dtype=np.float32)

# Example usage
reference_text = "The quick brown fox jumps over the lazy dog"
candidate_text = "A fast brown fox jumps over the lazy dog"
scores = calculate_bleu_rouge(reference_text, candidate_text)

print(scores)


[0.7259795 0.7777778 0.75      0.7777778]


In [24]:
# raw_model_metrics = df_test_all.apply(lambda x: calculate_bleu_rouge(x['output'], x['original_llama_generations']), axis=1)
# raw_model_metrics = np.array(raw_model_metrics).mean(-1)
# raw_model_metrics

array([0.01586311, 0.32068324, 0.08269039, 0.27278593], dtype=float32)

In [25]:
fine_tuned_model_metrics = df_test_all.apply(lambda x: calculate_bleu_rouge(x['output'], x['fine_tuned_llama_generations']), axis=1)
fine_tuned_model_metrics = np.array(fine_tuned_model_metrics).mean(-1)
fine_tuned_model_metrics

array([0.0198339 , 0.29949668, 0.08027118, 0.2574295 ], dtype=float32)

In [26]:
raw_model_metrics=np.array([0.012922,	0.307788,	0.077700,	0.256869])
df_metrics = pd.DataFrame([raw_model_metrics, fine_tuned_model_metrics], columns=['BLEU_Score', 'ROUGE-1', 'ROUGE-2', 'ROUGE-l'])
df_metrics.index  = ['Raw_llama2', 'Fine_tuned_llama2_batch*2']
df_metrics

,BLEU_Score,ROUGE-1,ROUGE-2,ROUGE-l
Raw_llama2,0.015863,0.320683,0.082690,0.272786
Fine_tuned_llama2_batch*2,0.019834,0.299497,0.080271,0.257430


In [25]:
# for i in range(len(df_test_all)):
#   row = df_test_all.iloc[i]
#   print(i)
#   print("label:")
#   print(row['output'])
#   print("generations:")
#   print(row['fine_tuned_llama_generations'])

In [ ]:
# for i in range(len(df_test_all)):
#   row = df_test_all.iloc[i]
#   print(i)
#   print("label:")
#   print(row['output'])
#   print("generations:")
#   print(row['original_llama_generations'])